
<p align="center"><img alt="udeA logo" height="150px" src="https://github.com/freddyduitama/images/blob/master/logo.png?raw=true"></p><h1><font color='0B5345'> <center>

Informática II

 </center></font></h1>
<h2><font color='0B5345'> <center>
Trabajo 1: Informática médica</center></font></h2>
<font  face="Courier New" size="3">

<p3><center><b><font color='0B5345' face="Lucida Calligraphy,Comic Sans MS,Lucida Console" size="5">Universidad de Antioquia - Bioingeniería</font></b> </center></p3>

En el ámbito de la investigación clínica, la gestión de imágenes médicas y su correspondiente metadata es fundamental para garantizar tanto la integridad del proceso investigativo como la privacidad de los pacientes. Las imágenes DICOM, que contienen datos detallados tanto clínicos como personales, requieren un manejo cuidadoso para evitar la divulgación de información identificable. 

Este proyecto tiene como objetivo desarrollar un sistema en Python que no solo gestione y anonimice estas imágenes, sino que también proporcione una estructura organizativa que identifique y catalogue imágenes de acuerdo a los pacientes y sus respectivos estudios médicos.



**Objetivo General:** Implementar un sistema basado en programación orientada a objetos para la gestión avanzada de imágenes DICOM, que incluya funcionalidades para agregar estudios a pacientes existentes, realizar consultas específicas, anonimizar estudios, convertir imágenes a formato NIfTI y manejar eficientemente la eliminación de estudios.

1. **Desarrollo de clases:**
   - Crear un programa que gestione la identificación única de pacientes y almacene sus estudios DICOM. Deberá detectar si un estudio pertenece a un nuevo paciente o a uno ya registrado, por medio del ID.
   - Implementar métodos para añadir, consultar y eliminar estudios, además de convertir imágenes DICOM a formato NIfTI.

2. **Sistema de consulta y reporte:**
   - Desarrollar funciones que permitan realizar consultas sobre los pacientes registrados y la cantidad de estudios que poseen, así como obtener detalles específicos de un estudio seleccionado, por medio del ID del paciente.
   - Incluir la capacidad de visualizar tanto las imágenes usando nilearn, como la información de los estudios (Fecha, modalidad, Descripción, dimensiones de la imagen) de un paciente específico y gestionar la eliminación de estudios.
   - Para la visualización con nilearn se debe poder elegir si se quiere visualizar un solo plano, los tres planos o en mosaico.

3. **Integración de anonimización y conversión de formatos:**
   - Integrar herramientas para la anonimización de estudios DICOM, asegurando que toda información identificable sea removida.
   - Facilitar la conversión de estudios DICOM a formato NIfTI para su uso en aplicaciones de análisis de imágenes más avanzadas.

**Metodología:**
- **Diagrama de clases:** Realizar las abstracción del problema y realizar el diagrama de clase asociado, incluya encapsulación, atributos y todos los métodos a utilizar.
- **Diseño de clases:** Utilizar la programación orientada a objetos para diseñar clases que encapsulen las funcionalidades necesarias, promoviendo la reutilización del código y la fácil mantenibilidad.
- **Pruebas de funcionalidad:** Realizar pruebas para cada funcionalidad implementada, para asegurar que el sistema funcione correctamente bajo diferentes escenarios.
- **Documentación:** Producir documentación completa del sistema.

Nota: Puede gestionar sus clases de la siguiente manera:

- Una clase `Estudio`  que maneje individualmente cada estudio DICOM, incluyendo funcionalidades para anonimizar y convertir formatos.
- Una clase `Paciente` que almacene una lista de estudios asociados a un paciente específico.
- Una clase `sistemaGestion` que gestione todos los pacientes y proporcione métodos para agregar estudios y consultar información.

El trabajo se realiza en grupos de 3 integrantes y debe ser entregado vía github, la documentación relacionada al trabajo debe estar consignada en el README del repositorio.

Adicionalmente se debe realizar un video de no más de 15 minutos donde expliquen las decisiones tomadas y realicen un ejemplo de implemtación de su programa.

¡EXITOS!

In [1]:
import pydicom
import dicom2nifti
import os
import nilearn
from nilearn import plotting


In [191]:
listaPacientes=[]
IDpacientes=[]

In [192]:
#Clase Estudio
class Estudio:
  def __init__(self): # idPaciente, fechaAdquisicion, modalidad, descripcion, imagenesDicom
    self.pacienteID = ''
    self.nombre=''
    self.fecha = ''
    self.modalidad = '' #Modalidad de la imagen(TC, IRM, etc)
    self.estudio = ''
    self.imagenesDicom = ''
    self.dimension=''

  def anonimizarEstudio(self,carpetaDICOM):
    '''
    Anonimiza el estudio eliminando o enmascarando la informacion sensible del paciente
    '''
    if os.listdir(carpetaDICOM):
      for dicomPath in os.listdir(carpetaDICOM):
        rutaCompleta = os.path.join(carpetaDICOM, dicomPath)
        dcm_data = pydicom.dcmread(rutaCompleta) #Lee el archivo DICOM
        dcm_data.PatientName = 'NAN' #Cambia el nombre del atributo PatientName
        dcm_data.PatientID = 'NAN' #Cambia el valor del atributo PatientID
        dcm_data.save_as(rutaCompleta) #Guarda los cambios en el archivo DICOM
      print('El estudio ha sido anonimizado')
    else:
      print('No se ha encontrado el estudio DICOM')
    
  def visualizarImagenes(self, carpetaNIFTI, modo='ortho'):
    '''
    Visualiza las imagenes NIFTI
    '''
    # try:
    if os.path.exists(carpetaNIFTI) and os.path.isfile(carpetaNIFTI):
      # os.listdir(carpetaNIFTI)
      if modo == 'ortho':
        plotting.plot_anat(carpetaNIFTI,
              display_mode="ortho")
      elif modo == 'y':
        plotting.plot_anat(carpetaNIFTI,
              display_mode="y")
      elif modo=='x':
        plotting.plot_anat(carpetaNIFTI,
              display_mode="x")
      elif modo == 'z':
        plotting.plot_anat(carpetaNIFTI,
              display_mode="z") 
      elif modo == 'mosaic':
        plotting.plot_anat(carpetaNIFTI,
              display_mode="mosaic")
      else:
        print(f'Modo de visualización no reconocido')
    else:
      print(f'No se encontró la carpeta {carpetaNIFTI}')
    
  def obtenerInfoEstudio(self,carpetaDICOM):
    try:
      os.listdir(carpetaDICOM)
      rutaIm=os.listdir(carpetaDICOM) 
      for i in rutaIm:  #En este ciclo solo recorremos una vez para extraer la metadata del DICOM
          rutaCompletaIm = os.path.join(carpetaDICOM, i) 
          dcm_data = pydicom.dcmread(rf'{rutaCompletaIm}')
          self.estudio=dcm_data.StudyDescription
          self.fecha=dcm_data.StudyDate
          self.modalidad=dcm_data.Modality
          self.dimension=dcm_data.pixel_array.shape
          print(f'''fechaAdquisicion': {self.fecha},
        'modalidad': {self.modalidad},
        'descripcion': {self.estudio},
        'dimensiones' : {self.dimension}''')
          break
    except:
      print('No se pudo cargar la metadata del estudio')

In [193]:
class Paciente:
    def __init__(self):   #Constructor de la clase para inicializar los atributos de instancia 
        self.pacienteID = ''
        self.nombre = ''  
    def obtener_nombres_pacientes(self,carpeta):
        if os.listdir(carpeta):    
            for nombre in os.listdir(carpeta):
                nombre=nombre.split('_')
                listaPacientes.append(nombre[1])
                IDpacientes.append(nombre[0])
            print(listaPacientes)
        else:
            print('Error, no hay carpetas de pacientes')

In [194]:
def validar(valor,tipo):
    '''
    Esta función tiene dos entradas, la primera es el input el cual se va a validar y la segunda es el tipo de input esperado, ya sea str o int
    '''
    while True:
        try:
            if tipo==int:
                if valor.isdigit():
                    return valor
                else:
                    print('-'*50)
                    print('Error, debe ingresar un número entero')
                    print('-'*50)
                    valor=input('Ingresar el valor nuevamente\n')
                    print('-'*50)
            elif tipo==str:
                if valor.isalpha():
                    return valor
                else:
                    print('-'*50)
                    print('Error, debe ingresar solo texto')
                    print('-'*50)
                    valor=input('Ingresar el valor nuevamente\n')
                    print('-'*50)
            else:
                return valor
        except:
            print('-'*50)
            print('Error, no ingresó el valor solicitado tipo'+str(tipo))
            Valor=input('Ingresar el valor nuevamente\n')
            print('-'*50)

In [195]:
class sistemaGestion():
    def __init__(self):
        self.carpetaPacientes=r'C:\Users\Pablo\Informática ll\git.repositorio\Inform-tica-ll\Unidad 3\Trabajo unidad 3\Pacientes' #Ruta de la carpeta de pacientes generada manualmente
        self.nombre = ''
        self.pacienteID=''
        self.carpetaPaciente=''
        self.carpetaEstudiosDICOM=''
        self.carpetaEstudiosNIFTI=''
    def agregarEstudio(self,carpetaDICOM): #Carpeta DICOM es la dirección donde se encuentra la carpeta YA DESCOMPRIMIDA
        try:
            os.listdir(carpetaDICOM)
            rutaIm=os.listdir(carpetaDICOM) 
            for i in rutaIm:  #En este ciclo solo recorremos una vez para extraer la metadata del DICOM
                rutaCompletaIm = os.path.join(carpetaDICOM, i) 
                dcm_data = pydicom.dcmread(rf'{rutaCompletaIm}')
                self.nombre=dcm_data.PatientName
                self.pacienteID=dcm_data.PatientID
                self.estudio=dcm_data.StudyDescription
                self.fecha=dcm_data.StudyDate
                self.modalidad=dcm_data.Modality
                break
            self.carpetaPaciente=f'{self.carpetaPacientes}\{self.pacienteID}_{self.nombre}' #Definimos la ruta ideal de la carpeta del paciente
            ruta=os.listdir(self.carpetaPacientes)
            if os.path.exists(os.path.join(self.carpetaPacientes, self.carpetaPaciente)):
                if os.path.isdir(os.path.join(self.carpetaPacientes, self.carpetaPaciente)):
                    print('Paciente ya existente') 
                    for i in ruta: #Recorremos la carpeta donde están todos los pacientes
                        rutaCompleta = os.path.join(self.carpetaPacientes, i) #Obtenemos la ruta de cada iteración
                        if rutaCompleta == f'{self.carpetaPacientes}\{self.pacienteID}_{self.nombre}': #Validamos si la ruta es la deseada a agregar el estudio
                            if not os.path.exists(f'{rutaCompleta}\EstudiosDICOM\{self.modalidad}_{self.estudio}_{self.fecha}'): #Se valida que el estudio no exista
                                self.carpetaEstudiosDICOM=f'{rutaCompleta}\EstudiosDICOM\{self.modalidad}_{self.estudio}_{self.fecha}' #Se define la carpeta donde se va a contener el estudio
                                self.carpetaEstudiosNIFTI=f'{rutaCompleta}\EstudiosNIFTI' #Se define la carpeta NIFTI del paciente
                                carpetaAuxiliar=f'{self.carpetaEstudiosNIFTI}\Auxiliar' 
                                os.mkdir(carpetaAuxiliar) #Creamos una carpeta auxiliar para cambiar el nombre del NIFTI
                                dicom2nifti.convert_directory(carpetaDICOM,carpetaAuxiliar)
                                for item in os.listdir(carpetaAuxiliar): #Pasamos el estudio a la carpeta deseada
                                    rutaCompleta = os.path.join(carpetaAuxiliar, item)
                                    os.rename(f'{rutaCompleta}',f'{self.carpetaEstudiosNIFTI}\{self.modalidad}_{self.estudio}_{self.fecha}.nii.gz') #Le cambiamos la dirección
                                    os.rmdir(carpetaAuxiliar) #Eliminamos la carpeta auxiliar
                                os.mkdir(self.carpetaEstudiosDICOM) #Creamos la carpeta con el estudio en DICOM
                                for item in rutaIm: #Recorremos la carpeta con los estudios DICOM y se llevan a otra carpeta con el nombre deseado
                                    item_path = os.path.join(carpetaDICOM, item)
                                    dcm_data=pydicom.dcmread(rf'{item_path}')
                                    dcm_data.save_as(f"{self.carpetaEstudiosDICOM}\{item}")
                                print('Estudio agregado correctamente')
                                break
                            else:
                                print('Error, estudio ya existente.')
                                break
                        else:
                            pass
            else:
                print('Paciente no existente, agregando paciente')
                self.agregarPacientes(self.nombre,self.pacienteID) #En caso de que el paciente no exista, se agrega con el método agregarPacientes
                self.agregarEstudio(carpetaDICOM) #Inmediatamente se vuelve a utilizar el método para guardar el estudio
        except:
            print('Error, carpeta no existente')
    def agregarPacientes(self,nombre,ID): #Nombre y ID son proporcionados por el usuario
        self.carpetaPacientes=r'C:\Users\Pablo\Informática ll\git.repositorio\Inform-tica-ll\Unidad 3\Trabajo unidad 3\Pacientes'
        self.nombre=nombre  
        self.pacienteID=ID
        self.carpetaPaciente=f'{self.carpetaPacientes}\{self.pacienteID}_{self.nombre}'
        if os.path.exists(os.path.join(self.carpetaPacientes, self.carpetaPaciente)): #Validamos que el paciente no exista por medio del ID
            if os.path.isdir(os.path.join(self.carpetaPacientes, self.carpetaPaciente)):
                print('Paciente ya existente')
        else: 
            self.carpetaEstudiosDICOM=f'{self.carpetaPaciente}\EstudiosDICOM'  #Definimos las carpetas donde se van a guardar los estudios del paciente
            self.carpetaEstudiosNIFTI=f'{self.carpetaPaciente}\EstudiosNIFTI' #Definimos las carpetas donde se van a guardar los estudios del paciente
            os.makedirs(self.carpetaPaciente, exist_ok=True) #Creamos la carpeta del paciente con su ID
            os.makedirs(self.carpetaEstudiosDICOM, exist_ok=True) #Generamos una carpeta del paciente para los estudios DICOM
            os.makedirs(self.carpetaEstudiosNIFTI, exist_ok=True) #Generamos una carpeta del paciente para los estudios NIFTI
            listaPacientes.append(self.nombre)
            IDpacientes.append(self.pacienteID)
            print('Paciente agregado correctamente')
    def consultarPacientes(self,nombre,ID): #Nombre y ID son proporcionados por el usuario
        self.carpetaPacientes=r'C:\Users\Pablo\Informática ll\git.repositorio\Inform-tica-ll\Unidad 3\Trabajo unidad 3\Pacientes'
        self.nombre=nombre  
        self.pacienteID=ID
        self.carpetaPaciente=f'{self.carpetaPacientes}\{self.pacienteID}_{self.nombre}'
        if os.path.exists(os.path.join(self.carpetaPacientes, self.carpetaPaciente)): #Verificamos si el paciente existe por medio del ID
            if os.path.isdir(os.path.join(self.carpetaPacientes, self.carpetaPaciente)):
                print('Paciente existente')
                print(f'El paciente {self.nombre} se encuentra en nuestro sistema de gestión.') #Le confiramos al usuario que está en el sistema
        else:
            print('Paciente no existente')
    def eliminarPaciente(self,nombre,ID): #Nombre y ID son proporcionados por el usuario
        self.pacienteID=ID
        self.carpetaPaciente=f'{self.carpetaPacientes}\{self.pacienteID}_{self.nombre}'
        if os.path.exists(os.path.join(self.carpetaPacientes, self.carpetaPaciente)): #Verificamos que el paciente exista por medio del ID
            if os.path.isdir(os.path.join(self.carpetaPacientes, self.carpetaPaciente)):
                self.nombre=nombre
                for item in os.listdir(self.carpetaPaciente): #Hacemos un ciclo para eliminar todo lo que está en la carpeta del paciente
                    item_path = os.path.join(self.carpetaPaciente, item)
                    if os.path.isdir(item_path):
                        os.rmdir(item_path)
                    else:
                        os.remove(item_path)
                os.rmdir(self.carpetaPaciente) #Cuando la carpeta está vacía, la eliminamos
                print('Paciente eliminado correctamente')
        else:
            print('Paciente no existente')
    def buscarEstudio(self,nombre,ID): #ID es proporcionado por el usuario
            self.pacienteID=ID
            self.nombre=nombre
            self.carpetaPaciente=f'{self.carpetaPacientes}\{self.pacienteID}_{self.nombre}'
            if os.path.exists(os.path.join(self.carpetaPacientes, self.carpetaPaciente)): #Verificamos que el paciente exista por medio del ID
                if os.path.isdir(os.path.join(self.carpetaPacientes, self.carpetaPaciente)):
                    ruta=f'{self.carpetaPaciente}\EstudiosDICOM' 
                    i=0
                    if os.listdir(ruta): #Validamos que el paciente tenga al menos un estudio
                        print('Estos son los estudios del paciente:')
                        for item in os.listdir(ruta):
                            i+=1
                            print(f'{i}. {item}') #Le mostramos al usuario todos sus estudios con sus nombres (que eson entendibles porque tienen fecha y descripción)
                        opcion=input('Ingrese la opción a la que desea ingresar')
                        opcion=validar(opcion,int) #Validamos con la función validar
                        if int(opcion)>i: #Verificamos que la opción ingresada no sea mayor que la posible
                            print('Opción no válida')
                            self.buscarEstudio(ID)
                        else:
                            i=0
                            for u in os.listdir(ruta):
                                i+=1
                                if int(opcion)==i:
                                    print('La ruta de acceso al estudio DICOM del paciente es:')
                                    print(f'{self.carpetaPaciente}\EstudiosDICOM\{u}') #Imprimimos la ruta DICOM de acceso al estudio para que sea visualizada en otro método
                                    print('La ruta de acceso al estudio NIFTI del paciente es:')
                                    print(f'{self.carpetaPaciente}\EstudiosNIFTI\{u}.nii.gz')#Imprimimos la ruta NIFTI de acceso al estudio para que sea visualizada en otro método
                                    break
                    else:
                        print('El paciente no tiene estudios')
    def eliminarEstudio(self,nombre,ID): #ID es proporcionado por el usuario
            self.pacienteID=ID
            self.nombre=nombre
            self.carpetaPaciente=f'{self.carpetaPacientes}\{self.pacienteID}_{self.nombre}'
            if os.path.exists(os.path.join(self.carpetaPacientes, self.carpetaPaciente)): #Validamos que el paciente exista
                if os.path.isdir(os.path.join(self.carpetaPacientes, self.carpetaPaciente)):
                    ruta=f'{self.carpetaPaciente}\EstudiosDICOM'
                    i=0
                    if os.listdir(ruta): #Validamos que el paciente tenga al menos un estudio
                        print('Estos son los estudios del paciente:')
                        for item in os.listdir(ruta):
                            i+=1
                            print(i)
                            print(f'{i}. {item}') #Le mostramos al usuario todos sus estudios con sus nombres (que eson entendibles porque tienen fecha y descripción)
                        opcion=input('Ingrese la opción a la que desea ingresar')
                        opcion=validar(opcion,int)
                        if int(opcion)>i:
                            print('Opción no válida')
                        else:
                            i=0
                            for u in os.listdir(ruta):
                                i+=1
                                if int(opcion)==i:
                                    rutaDICOM=f'{self.carpetaPaciente}\EstudiosDICOM\{u}' #Definimos las rutas de las carpetas a eliminar
                                    rutaNIFTI=f'{self.carpetaPaciente}\EstudiosNIFTI\{u}.nii.gz'
                                    for item in os.listdir(rutaDICOM): #Vaciaamos la carpeta DICOM
                                        item_path = os.path.join(rutaDICOM, item)
                                        if os.path.isdir(item_path):
                                            os.rmdir(item_path)
                                        else:
                                            os.remove(item_path)
                                    os.rmdir(rutaDICOM) #Eliminamos la carpeta vacía DICOM
                                    os.remove(rutaNIFTI) #Eliminamos la carpeta comprimida NIFTI
                                    print('Estudio eliminado correctamente')
                                    break
                    else:
                        print('El paciente no tiene estudios')
                else:
                    print('El paciente no existe')
            else:
                print('El paciente no existe')
    def infoEstudio(self,carpetaDICOM):
        try:
            os.listdir(carpetaDICOM)
            estudio=Estudio() #Asosiación con la clase Estudio
            estudio.obtenerInfoEstudio(carpetaDICOM) #Asosiación
        except:
            print('No se pudo obtener la información del estudio, carptea no existente')
    def visualizar(self,carpetaEstudiosNifti):
        estudio=Estudio() #Asosiación con la clase Estudio
        modo=input('Ingrese el modo de visualización deseado:\n1. ortho\n2. Un solo plano\n3. mosaic')
        modo=validar(modo,int)
        if modo=='1':
            MODO='ortho'
            estudio.visualizarImagenes(carpetaEstudiosNifti,MODO)
        elif modo=='2':
            MODO=input('Ingrese la opción de plano que desea ver\n1. x\n2. y\n3. z\n')
            MODO=validar(MODO,int)
            if MODO=='1':
                modo='x'
                estudio.visualizarImagenes(carpetaEstudiosNifti,modo)
            elif MODO=='2':
                modo='y'
                estudio.visualizarImagenes(carpetaEstudiosNifti,modo)
            elif MODO=='z':
                modo='z'
                estudio.visualizarImagenes(carpetaEstudiosNifti,modo)
            else:
                print('Opción no válida')
        elif modo=='3':
            MODO='mosaic'
            estudio.visualizarImagenes(carpetaEstudiosNifti,MODO)
        else:
            print('Opción no válida')
    def anonimizarEstudio(self,carpetaDicom):
        estudio=Estudio() #Asosiación con la clase Estudio
        estudio.anonimizarEstudio(carpetaDicom)
    def listaPacientes(self):
        paciente=Paciente() #Asosiación con la clase Paciente
        paciente.obtener_nombres_pacientes(self.carpetaPacientes)

In [ ]:
sistema=sistemaGestion()
while True:
    menu=input('Ingrese la opción a la que desea ingresar:\n1. Agregar paciente\n2. Consultar paciente\n3. Eliminar paciente\n4. Cargar estudio\n5. Buscar estudio\n6. Eliminar estudio\n7. Obtener información del estudio\n8. Visualizar imagen\n9. Obtener lista de pacientes\n10. Anonimizar estudio\n11. Salir')
    menu=validar(menu,int)
    if menu=='1':
        nombre=input('Ingrese el nombre del paciente\n')
        nombre=validar(nombre,str)
        ID=input('Ingrese el ID del paciente\n')
        sistema.agregarPacientes(nombre,ID)
    elif menu=='2':
        nombre=input('Ingrese el nombre del paciente\n')
        nombre=validar(nombre,str)
        ID=input('Ingrese el ID del paciente\n')
        sistema.consultarPacientes(nombre,ID)
    elif menu=='3':
        nombre=input('Ingrese el nombre del paciente\n')
        nombre=validar(nombre,str)
        ID=input('Ingrese el ID del paciente\n')
        sistema.eliminarPaciente(nombre,ID)
    elif menu=='4':
        carpetaDICOM=input('Ingrese la ruta de la carpeta DICOM que desea cargar\n')
        sistema.agregarEstudio(carpetaDICOM)
    elif menu=='5':
        nombre=input('Ingrese el nombre del paciente\n')
        nombre=validar(nombre,str)
        ID=input('Ingrese el ID del paciente\n')
        sistema.buscarEstudio(nombre,ID)
    elif menu=='6':
        nombre=input('Ingrese el nombre del paciente\n')
        nombre=validar(nombre,str)
        ID=input('Ingrese el ID del paciente\n')
        sistema.eliminarEstudio(nombre,ID)
    elif menu=='7':
        carpetaDICOM=input('Ingrese la ruta de la carpeta DICOM que desea cargar\n')
        sistema.infoEstudio(carpetaDICOM)
    elif menu=='8':
        carpetaNifti=input('Ingrese la dirección del estudio NIFTI')
        sistema.visualizar(carpetaNifti)
    elif menu=='9':
        sistema.listaPacientes()
    elif menu=='10':
        carpetaDICOM=input('Ingrese la ruta de la carpeta DICOM que desea anonimizar\n')
        sistema.anonimizarEstudio(carpetaDICOM)
    elif menu=='11':
        break
    else:
        print('Opción no válida')
print('Gracias por utilizar nuestro sistema de gestión.')